In [ ]:
from IPython.display import display
import pandas as pd
import numpy as np

In [ ]:
tfr = pd.read_csv("FMCSA_CENSUS1_2025Feb.txt", sep = ",", encoding="windows-1252", iterator = True, chunksize = 100000)   
df = pd.concat(tfr, ignore_index = True)
print(display(df.head()))

In [ ]:
#df = df.apply(lambda x: pd.Series(x.dropna().values))

print(df.columns)
print(display(df.head()))

Index(['DOT_NUMBER', 'LEGAL_NAME', 'DBA_NAME', 'CARRIER_OPERATION', 'HM_FLAG',
       'PC_FLAG', 'PHY_STREET', 'PHY_CITY', 'PHY_STATE', 'PHY_ZIP',
       'PHY_COUNTRY', 'MAILING_STREET', 'MAILING_CITY', 'MAILING_STATE',
       'MAILING_ZIP', 'MAILING_COUNTRY', 'TELEPHONE', 'FAX', 'EMAIL_ADDRESS',
       'MCS150_DATE', 'MCS150_MILEAGE', 'MCS150_MILEAGE_YEAR', 'ADD_DATE',
       'OIC_STATE', 'NBR_POWER_UNIT', 'DRIVER_TOTAL', 'RECENT_MILEAGE',
       'RECENT_MILEAGE_YEAR', 'VMT_SOURCE_ID', 'PRIVATE_ONLY',
       'AUTHORIZED_FOR_HIRE', 'EXEMPT_FOR_HIRE', 'PRIVATE_PROPERTY',
       'PRIVATE_PASSENGER_BUSINESS', 'PRIVATE_PASSENGER_NONBUSINESS',
       'MIGRANT', 'US_MAIL', 'FEDERAL_GOVERNMENT', 'STATE_GOVERNMENT',
       'LOCAL_GOVERNMENT', 'INDIAN_TRIBE', 'OP_OTHER'],
      dtype='object')


,DOT_NUMBER,LEGAL_NAME,DBA_NAME,CARRIER_OPERATION,HM_FLAG,PC_FLAG,PHY_STREET,PHY_CITY,PHY_STATE,PHY_ZIP,...,PRIVATE_PROPERTY,PRIVATE_PASSENGER_BUSINESS,PRIVATE_PASSENGER_NONBUSINESS,MIGRANT,US_MAIL,FEDERAL_GOVERNMENT,STATE_GOVERNMENT,LOCAL_GOVERNMENT,INDIAN_TRIBE,OP_OTHER
0,1,FEDERAL MOTOR CARRIER SAFETY ADMINISTRATION,FMCSA TECHNOLOGY DIVISION,A,N,N,1200 NEW JERSEY AVENUE SE,WASHINGTON,DC,20590,...,N,N,N,N,N,Y,N,N,N,AUTOMATED VEHICLE RESEARCH & TESTING
1,10000,POWELL DISTRIBUTING CO INC,JAMES KILLINGSWORTH TRUCKING,A,N,N,9125 N BURRAGE,PORTLAND,OR,97217,...,Y,N,N,N,N,N,N,N,N,N
2,1000000,JAMES EARL KILLINGSWORTH JR,PARK ADAM TRANSPORTATION,A,N,N,15 GASLINE ROAD,PHENIX CITY,AL,36870,...,N,N,N,N,N,N,N,N,N,N
3,1000002,NEW JERSEY BOOM & ERECTORS INC,WESTSIDE BUILDING SUPPLY,A,N,N,405 INDUSTRIAL PARK DRIVE,MOUNT POCONO,PA,18344,...,Y,N,N,N,N,N,N,N,N,N
4,1000004,RAY TRUCKING LLC,ALS AUTO SALVAGE,C,N,N,218 PEACHTREE ST,WARWICK,GA,31796,...,N,N,N,N,N,N,N,N,N,COTTON


None
